In [1]:
import warnings
warnings.filterwarnings("ignore")
from itertools import combinations

import gc
import os
import sys
import json
import xgboost

import numpy as np
import pandas as pd
import numba as nb
import polars as pl

from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# Functions

In [2]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage of dataframe is {start_mem:.2f} MB')
    
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float32)
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Memory usage after optimization is: {end_mem:.2f} MB')
    
    decrease = 100 * (start_mem - end_mem) / start_mem
    print(f'Decreased by {decrease:.2f}%')
    
    return df


@nb.jit(nopython=False, parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))
    for i in range(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            
            if mid_val == min_val:
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)
    return imbalance_features


@nb.jit(nopython=False,fastmath=True)
def calculate_triplet_imbalance(price, df):
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]
    features_array = compute_triplet_imbalance(df_values, comb_indices)
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)
    return features


@nb.jit(nopython=False, parallel=True)
def calculate_weighted_wap(df):
    df["stock_weights"] = df["stock_id"].map(weights)
    df["weighted_wap"] = df["stock_weights"] * df["wap"]
    df['wap_momentum'] = df.groupby('stock_id')['weighted_wap'].pct_change(periods=6)
    return df


@nb.jit(nopython=False, parallel=True)
def calculate_rolling_mean_and_std_expressions(df):
    # Convert from pandas to Polars
    pl_df = pl.from_pandas(df)

    # Define the windows and columns for which you want to calculate the rolling statistics
    windows = [3, 5, 10]
    columns = ['ask_price', 'bid_price', 'ask_size', 'bid_size']

    # prepare the operations for each column and window
    group = ["stock_id"]
    expressions = []

    # Loop over each window and column to create the rolling mean and std expressions
    for window in windows:
        for col in columns:
            rolling_mean_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_mean(window)
                .over(group)
                .alias(f'rolling_diff_{col}_{window}')
            )

            rolling_std_expr = (
                pl.col(f"{col}_diff_{window}")
                .rolling_std(window)
                .over(group)
                .alias(f'rolling_std_diff_{col}_{window}')
            )

            expressions.append(rolling_mean_expr)
            expressions.append(rolling_std_expr)

    # Run the operations using Polars' lazy API
    lazy_df = pl_df.lazy().with_columns(expressions)

    # Execute the lazy expressions and overwrite the pl_df variable
    pl_df = lazy_df.collect()

    # Convert back to pandas if necessary
    df = pl_df.to_pandas()
    
    return df

def feat_eng(df, epsilon=1e-8):   
    cols = [c for c in df.columns if c not in ['row_id', 'time_id', 'currently_scored']]
    df = df[cols]
    
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    
    df = calculate_weighted_wap(df)
        
    # Adding some features
    df["dow"] = df["date_id"] % 5  # Day of the week
    df["seconds"] = df["seconds_in_bucket"] % 60  
    df["minute"] = df["seconds_in_bucket"] // 60
    df['time_to_market_close'] = 540 - df['seconds_in_bucket']
    
    df['imbalance_buy_flag'] = np.where(df['imbalance_buy_sell_flag']==1, 1, 0) 
    df['imbalance_sell_flag'] = np.where(df['imbalance_buy_sell_flag']==-1, 1, 0) 
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    df['imbalance_ratio'] = df.eval('imbalance_size / (matched_size + @epsilon)')
    
    df['imb_s1'] = df.eval('(bid_size - ask_size) / (bid_size + ask_size + @epsilon)')
    df['imb_s2'] = df.eval('(imbalance_size - matched_size) / (matched_size + imbalance_size + @epsilon)')
    
    df["imbalance_momentum"] = df.groupby(['stock_id'])['imbalance_size'].diff(periods=1) / df['matched_size']
    df["matched_imbalance"] = df.eval("(imbalance_size - matched_size)/(matched_size + imbalance_size)")
        
    df['ask_x_size'] = df.eval('ask_size * ask_price')
    df['bid_x_size'] = df.eval('bid_size * bid_price')
        
    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size'] 
    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])
    df['price_pressure'] = df['imbalance_size'] * (df['ask_price'] - df['bid_price'])
    df['depth_pressure'] = (df['ask_size'] - df['bid_size']) * (df['far_price'] - df['near_price'])
    df['relative_spread'] = (df['ask_price'] - df['bid_price']) / df['wap']
    df['harmonic_imbalance'] = df.eval('2 / ((1 / bid_size) + (1 / ask_size))')
    
    df["price_spread"] = df["ask_price"] - df["bid_price"]
    df["spread_intensity"] = df.groupby(['stock_id'])['price_spread'].diff()
    
    df["volume"] = df.eval("ask_size + bid_size")
    df["mid_price"] = df.eval("(ask_price + bid_price) / 2")
    df['mid_price_movement'] = df['mid_price'].diff(periods=5).apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    df['mid_price*volume'] = df['mid_price_movement'] * df['volume']
    
    # Calculate diff features for specific columns
    for col in ['imb_s1', 'ask_price', 'bid_price', 'wap', 'ask_size', 'bid_size', 'weighted_wap', 'price_spread']:
        for window in [2, 3, 5, 10]:
            df[f"{col}_diff_{window}"] = df.groupby(["stock_id", "date_id"])[col].diff(window)
            
    df = calculate_rolling_mean_and_std_expressions(df)
            
    for col in ['imb_s1', 'ask_price', 'bid_price', 'wap']:
        df[f"{col}_gain"] = df[f"{col}_diff_{window}"].where(df[f"{col}_diff_{window}"] > 0, 0)
        df[f"{col}_loss"] = df[f"{col}_diff_{window}"].where(df[f"{col}_diff_{window}"] < 0, 0)
        df[f"{col}_gain_avg"] = df[f"{col}_gain"].rolling(window=5).mean()
        df[f"{col}_loss_avg"] = df[f"{col}_loss"].rolling(window=5).mean()
                    
    for col in ['imb_s1', 'ask_price', 'bid_price', 'wap']:
        df[f"{col}_cummax"] = df.groupby(["stock_id", "date_id"])[col].cummax() 
        df[f"{col}_cummin"] = df.groupby(["stock_id", "date_id"])[col].cummin() 
        df[f"{col}_min_max_norm"] = df.eval(f'({col} - {col}_cummin) / ({col}_cummax - {col}_cummin)')
        
    columns_to_keep = [col for col in df.columns if 'cummax' not in col]
    df = df[columns_to_keep]
    columns_to_keep = [col for col in df.columns if 'cummin' not in col]
    df = df[columns_to_keep]
        
    # Bollinger Bands 
    periods = [3, 5, 7, 9]
    for col in ['imb_s1', 'ask_price', 'bid_price', 'wap']:
        grouped = df.groupby(["stock_id", "date_id"])[col]
        for p in periods: 
            grouped_std = grouped.rolling(window=p).std().reset_index(drop=True)            
            df[f"{col}_bollinger_upper_{p}"] = df[f"{col}"] + 2 * grouped_std
            df[f"{col}_bollinger_lower_{p}"] = df[f"{col}"] - 2 * grouped_std
            
    for c in [['ask_price', 'bid_price', 'wap', 'reference_price'], sizes]:
        triplet_feature = calculate_triplet_imbalance(c, df)
        df[triplet_feature.columns] = triplet_feature.values
        
    for c in combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = df[f'{c[0]}'] - df[f'{c[1]}']
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]} - {c[1]}) / ({c[0]} + {c[1]} + @epsilon)')
        df[f'{c[0]}_{c[1]}_urgency'] = df[f'{c[0]}_minus_{c[1]}'] * df['imb_s1']
    
    columns_to_keep = [col for col in df.columns if 'minus' not in col]
    df = df[columns_to_keep]
    
    for func in ["mean", "std", "skew", "kurt"]:
        df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
        df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
        
    for window in [3,5,10]:
        df[f'price_change_diff_{window}'] = df[f'bid_price_diff_{window}'] - df[f'ask_price_diff_{window}']
        df[f'size_change_diff_{window}'] = df[f'bid_size_diff_{window}'] - df[f'ask_size_diff_{window}']
        
    # Add other features              
    for i in range(len(prices)-1):
        p1 = prices[i]
        for j in range(i+1, len(prices)):
            p2 = prices[j]
            df[f'{p1}_{p2}_pct_change'] = ((df[p1] - df[p2]) / (df[p2] + epsilon)) * 100
            
    for key, value in global_stock_id_feats.items():
        df[f"global_{key}"] = df["stock_id"].map(value.to_dict())
        
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['global_median_vol'], 1, 0) 
    
    # Reduce memory usage
    df = reduce_mem_usage(df)
    
    # Run garbage collector
    gc.collect()
    return df

weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
]
weights = {int(k):v for k,v in enumerate(weights)}


def zero_sum(prices, volumes):
    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step
    return out


def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return w

# Dataset

In [3]:
# Load the dataset
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')

# Drop nan on target column
train.dropna(subset=['target'], inplace=True)

global_stock_id_feats = {
        "median_vol": train.groupby("stock_id")["bid_size"].median() + train.groupby("stock_id")["ask_size"].median(),
        "std_size": train.groupby("stock_id")["bid_size"].std() + train.groupby("stock_id")["ask_size"].std(),
        "ptp_size": train.groupby("stock_id")["bid_size"].max() - train.groupby("stock_id")["bid_size"].min(),
        "median_price": train.groupby("stock_id")["bid_price"].median() + train.groupby("stock_id")["ask_price"].median(),
        "std_price": train.groupby("stock_id")["bid_price"].std() + train.groupby("stock_id")["ask_price"].std(),
        "ptp_price": train.groupby("stock_id")["bid_price"].max() - train.groupby("stock_id")["ask_price"].min()
}

# Run garbage collector
gc.collect()

# Drop other relevant nan
train.dropna(subset=['bid_price', 'ask_price'], inplace=True)

# Reset the index
train.reset_index(drop=True)

# Apply features engineering
train = feat_eng(train)

# Get X ed Y
y = train['target']
X = train.drop(columns='target')

# Get date ids
date_ids = X['date_id'].values
X = X.drop(columns='date_id')

# Get feature names
feature_name = list(X.columns)

# Run garbage collector
gc.collect()

Memory usage of dataframe is 9031.17 MB
Memory usage after optimization is: 2707.35 MB
Decreased by 70.02%


0

# Model

In [4]:
num_folds = 5
fold_size = 480 // num_folds
gap = 5

stopping_rounds = 128
log_evaluation_periods = 2

xgboost_params = {
    'n_estimators': 6400,
    'booster': 'gbtree',
    'device': 'cuda',
    'verbosity': 2, 
    'learning_rate': 0.0085,
    'max_depth': 16,
    'subsample': 0.75,
    'colsample_bytree': 0.80,
    #'sampling_method': 'gradient_based', # 'gradient_based'
    'tree_method': 'hist',
    'grow_policy': 'depthwise',
    'max_leaves': 256,
    'max_bin': 3800,
    'objective':'reg:absoluteerror', 
    'reg_lambda': 0.00001,
    'reg_alpha': 0.00001
}

# Define lists
models = []
scores = []

# Run garbage collector
gc.collect()

# Loop over data
for i in range(num_folds):
    print(f"\n\nFold {i+1} Model Training")
    start = i * fold_size
    end = start + fold_size
    if i < num_folds - 1:  # No need to purge after the last fold
        purged_start = end - 2
        purged_end = end + gap + 2
        train_indices = (date_ids >= start) & (date_ids < purged_start) | (date_ids > purged_end)
    else:
        train_indices = (date_ids >= start) & (date_ids < end)
    
    val_indices = (date_ids >= end) & (date_ids < end + fold_size)
    
    # Get the data for the current fold
    X_fold_train = X[train_indices]
    y_fold_train = y[train_indices]
    X_fold_valid = X[val_indices]
    y_fold_valid = y[val_indices]
    
    # Define a Xgboost regressor model for the current fold
    xgboost_model = XGBRegressor(**xgboost_params)
    
    # Train the Xgboost model for the current fold
    xgboost_model.fit(
        X_fold_train[feature_name],
        y_fold_train,
        eval_set=[(X_fold_valid[feature_name], y_fold_valid)],
        callbacks=[
            xgboost.callback.EvaluationMonitor(period=log_evaluation_periods),
            xgboost.callback.EarlyStopping(rounds=stopping_rounds)
        ]
    )
    
    # Fill the model list
    models.append(xgboost_model)
        
    # Free up memory by deleting fold specific variables
    del X_fold_train, y_fold_train, X_fold_valid, y_fold_valid
    
    # Run garbage collector
    gc.collect()

# Calculate the average best iteration from all regular folds
average_best_iteration = int(np.mean([model.best_iteration for model in models]))

# Update the lgb_params with the average best iteration
final_model_params = xgboost_params.copy()
# final_model_params['n_estimators'] = average_best_iteration

print(f"\nTraining final model with average best iteration: {average_best_iteration}")

# Train the final model on the entire dataset
final_model = XGBRegressor(**final_model_params)
final_model.fit(
    X[feature_name],
    y,
    callbacks=[
        xgboost.callback.EvaluationMonitor(period=log_evaluation_periods),
    ],
)

# Append the final model to the list of models
models.append(final_model)

# Run garbage collector
gc.collect()



Fold 1 Model Training
[0]	validation_0-mae:7.24094
[0]	validation_0-mae:7.24094
[1]	validation_0-mae:7.23853
[2]	validation_0-mae:7.23614
[2]	validation_0-mae:7.23614
[3]	validation_0-mae:7.23377
[4]	validation_0-mae:7.23145
[4]	validation_0-mae:7.23145
[5]	validation_0-mae:7.22922
[6]	validation_0-mae:7.22704
[6]	validation_0-mae:7.22704
[7]	validation_0-mae:7.22483
[8]	validation_0-mae:7.22260
[8]	validation_0-mae:7.22260
[9]	validation_0-mae:7.22044
[10]	validation_0-mae:7.21832
[10]	validation_0-mae:7.21832
[11]	validation_0-mae:7.21624
[12]	validation_0-mae:7.21422
[12]	validation_0-mae:7.21422
[13]	validation_0-mae:7.21217
[14]	validation_0-mae:7.21015
[14]	validation_0-mae:7.21015
[15]	validation_0-mae:7.20817
[16]	validation_0-mae:7.20622
[16]	validation_0-mae:7.20622
[17]	validation_0-mae:7.20430
[18]	validation_0-mae:7.20241
[18]	validation_0-mae:7.20241
[19]	validation_0-mae:7.20053
[20]	validation_0-mae:7.19869
[20]	validation_0-mae:7.19869
[21]	validation_0-mae:7.19686
[

1631

# Submission

In [5]:
# Init
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

# Init a counter
counter = 0

# To clip predictions
y_min, y_max = -64, 64

# Init an empty dataframe
cache = pd.DataFrame()

# Weights for each fold model
# model_weights = [1/len(models)] * len(models) 
model_weights = weighted_average(models)

for (test, revealed_targets, sample_prediction) in iter_test:       
    # Add data to the chache dataset
    cache = pd.concat([cache, test], ignore_index=True, axis=0)
    if counter > 0:
        cache = cache.groupby(['stock_id']).tail(21).sort_values(by=['date_id', 'seconds_in_bucket', 'stock_id']).reset_index(drop=True)
        
    # First iteration 
    if test.currently_scored.iloc[0]==False:
        sample_prediction['target'] = 0
        env.predict(sample_prediction)
        continue

    # Prepare the data
    features = feat_eng(cache)[-len(test):]
    features = features.drop(columns='date_id')

    # Generate predictions for each model and calculate the weighted average
    lgb_predictions = np.zeros(len(test))
    for model, weight in zip(models, model_weights):
        lgb_predictions += weight * model.predict(features)

    # Zero sum
    #lgb_predictions = zero_sum(lgb_predictions, test['bid_size'] + test['ask_size'])
    
    lgb_predictions_zero_sum = zero_sum(lgb_predictions, test['bid_size'] + test['ask_size'])
    lgb_predictions_mean = lgb_predictions - np.mean(lgb_predictions)
    lgb_predictions = (lgb_predictions_zero_sum + lgb_predictions_mean) / 2
    
    # Clip the prediction
    clipped_predictions = np.clip(lgb_predictions, y_min, y_max)

    # Predict
    sample_prediction['target'] = clipped_predictions
    env.predict(sample_prediction)
    
    # Update the counter
    counter += 1
    
    # Run garbage collector
    gc.collect()

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
